In [2]:
import pandas as pd
import os
import sys

sys.path.append("../..")
from common.utils import timestep_csv_names
from common.constants import DATAFOLDER

root_dir = os.path.join(DATAFOLDER.data_root_path, "one_day_data")

In [9]:
import json
f = open("./valid_dataset.json")
valid_set = json.load(f)
valid_dates = []
for case_name in valid_set.keys():
    for sample_name in valid_set[case_name].keys():
        date = valid_set[case_name][sample_name]["0"]["date"]
        valid_dates.append(date)

print(valid_dates)

['2020-10-12', '2020-09-14', '2020-08-07', '2020-07-04', '2019-10-04', '2020-10-12']


In [14]:
_timestep_csv_names = timestep_csv_names(delta=10)

dates, rain_amounts, start_times, end_times = [], [], [], []
for year in os.listdir(root_dir):
    for month in os.listdir(root_dir + f"/{year}"):
        for date in os.listdir(root_dir + f"/{year}/{month}"):
            if date not in valid_dates:
                minimum_data_length = 40
                max_rainfall = 0
                start_time = None
                end_time = None
                for csv_filename in _timestep_csv_names:
                    csv_path = os.path.join(
                            root_dir,
                            year, month, date, csv_filename
                        )
                    parquet_path = csv_path.replace(".csv", ".parquet.gzip")
                    parquet_df = pd.read_parquet(parquet_path, engine="pyarrow")

                    if parquet_df["hour-rain"].max() >= 5:
                        if parquet_df["hour-rain"].max() > max_rainfall:
                            max_rainfall = parquet_df["hour-rain"].max()
                        if start_time is None:
                            start_time = csv_filename.replace(".csv", "")

                    if start_time is not None and parquet_df["hour-rain"].max() < 5:
                        end_time = csv_filename.replace(".csv", "")
                        if _timestep_csv_names.index(end_time+".csv") - _timestep_csv_names.index(start_time+".csv") > 7:
                            dates.append(date)
                            rain_amounts.append(max_rainfall)
                            start_times.append(start_time)
                            end_times.append(end_time)
                            
                        start_time = None
                        end_time = None
                        max_rainfall = 0

                    if len(parquet_df.index) < minimum_data_length:
                        minimum_data_length = len(parquet_df.index)

                if minimum_data_length > 0 and start_time is not None and end_time is None:
                    end_time = csv_filename.replace(".csv", "")
                    if _timestep_csv_names.index(end_time+".csv") - _timestep_csv_names.index(start_time+".csv") > 7:
                        dates.append(date)
                        rain_amounts.append(max_rainfall)
                        start_times.append(start_time)
                        end_times.append(end_time)

df = pd.DataFrame({"date": dates, "rain": rain_amounts, "start_time": start_times, "end_time": end_times})
df = df.sort_values(by="rain", ascending=False)
df.reset_index()
df.head()

,date,rain,start_time,end_time
66,2020-08-01,88.0,3-40,7-10
1,2020-07-27,80.0,10-50,13-40
20,2020-07-12,76.5,21-10,23-20
152,2020-05-28,65.0,5-20,7-30
131,2020-10-24,62.5,9-50,15-40


In [15]:
df.to_csv("train_dataset.csv")

In [18]:
for date in df["date"]:
    assert date not in valid_dates